In [ ]:
import streamlit as st
import pandas as pd
from datetime import datetime

st.title("📅 Client Hours Formatter")

st.write("Upload a file or paste your data below in the format: `Harmony - 2`")

date_input = st.date_input("Pick a date (for pasted input)", value=None)
raw_text = st.text_area("Paste daily entries here (one per line)", height=200)

uploaded_file = st.file_uploader("OR upload a CSV with columns: Date, Client, Hours")

def parse_text_block(raw_text, date_input):
    rows = []
    if not date_input or not raw_text:
        return pd.DataFrame()

    lines = raw_text.strip().split('\n')
    for line in lines:
        if '-' in line:
            parts = line.split('-')
            client = parts[0].strip()
            hrs = float(parts[1].strip())
            day = f"{date_input.day} {date_input.strftime('%a').upper()}"
            rows.append([client, hrs, day])
    return pd.DataFrame(rows, columns=["CLIENT", "HRS", "DAY"])

if raw_text and date_input:
    df = parse_text_block(raw_text, date_input)
    st.success("Parsed successfully!")
    st.dataframe(df.sort_values(by=["CLIENT", "DAY"]))

elif uploaded_file:
    try:
        df = pd.read_csv(uploaded_file)
        df['DAY'] = pd.to_datetime(df['Date']).apply(lambda x: f"{x.day} {x.strftime('%a').upper()}")
        df.rename(columns={"Client": "CLIENT", "Hours": "HRS"}, inplace=True)
        df_final = df[["CLIENT", "HRS", "DAY"]].sort_values(by=["CLIENT", "DAY"])
        st.success("Uploaded and converted successfully!")
        st.dataframe(df_final)
    except Exception as e:
        st.error(f"Error reading file: {e}")